In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
def pre_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rows, cols = gray.shape[0], gray.shape[1]
    pre_lbp = np.zeros((rows, cols), dtype=np.uint8)  
    return gray, rows, cols, pre_lbp


def calculate_lbp_clockwise(image):
    gray, rows, cols, lbp_clockwise = pre_lbp(image)
    for i in range(1, rows - 1):  # Hindari satu piksel di tepi atas dan bawah
        for j in range(1, cols - 1):  # Hindari satu piksel di tepi kiri dan kanan
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan searah jarum jam mulai dari 3 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j + 1], gray[i, j + 1],
                         gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1], gray[i - 1, j - 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            lbp_clockwise[i - 1, j - 1] = code
    return lbp_clockwise

def calculate_lbp_anticlockwise(image):
    gray, rows, cols, lbp_anticlockwise = pre_lbp(image)
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan berlawanan arah jarum jam mulai dari 9 o'clock
            neighbors = [gray[i - 1, j - 1], gray[i - 1, j], gray[i - 1, j + 1],
                         gray[i, j + 1], gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            lbp_anticlockwise[i - 1, j - 1] = code
    
    return lbp_anticlockwise

    
def calculate_ncdblbp_clockwise(image):
    gray, rows, cols, ncdlbp_clockwise = pre_lbp(image)
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan searah jarum jam mulai dari 3 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j + 1], gray[i, j + 1],
                         gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1], gray[i - 1, j - 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            ncdlbp_clockwise[i, j] = code
    
    return ncdlbp_clockwise


def calculate_ncdblbp_anticlockwise(image):
    gray, rows, cols, ncdlbp_anticlockwise = pre_lbp(image)
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan berlawanan arah jarum jam mulai dari 9 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j - 1], gray[i, j - 1],
                         gray[i + 1, j - 1], gray[i + 1, j], gray[i + 1, j + 1],
                         gray[i, j + 1], gray[i - 1, j + 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            ncdlbp_anticlockwise[i, j] = code
    
    return ncdlbp_anticlockwise


In [37]:
# Tentukan direktori awal yang ingin Anda iterasi
root_directory = 'dataset-fix'

data = []

# Iterasi semua folder, subfolder, dan file dalam direktori
for root, directories, files in os.walk(root_directory):
    # Cetak nama folder saat ini
    folder_name = os.path.basename(root)

    # Iterasi semua subfolder dalam folder saat ini (root)
    for directory in directories:
        subfolder_path = os.path.join(root, directory)
        print(directory)
        # Iterasi semua file dalam subfolder saat ini
        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)
            image_path = file_path
            image = cv2.imread(image_path)
            
            temp_lbp = calculate_ncdblbp_anticlockwise(image)
            temp_lbp = temp_lbp.flatten()
            temp_lbp = temp_lbp.astype(object)
            temp_lbp = np.insert(temp_lbp, 0, directory)
            data.append(temp_lbp)
            # print(f'{directory}/{file}')



Angry
Happy
Other
Sad


In [38]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40248,40249,40250,40251,40252,40253,40254,40255,40256,40257
0,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Tentukan threshold (50% dari panjang DataFrame)
threshold = len(df) * 0.7

# Menghapus kolom di mana lebih dari 50% nilai adalah NaN
df = df.dropna(axis=1, thresh=threshold)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11202,11203,11204,11205,11206,11207,11208,11209,11210,11211
0,Angry,0,0,0,0,0,0,0,0,0,...,39.0,255.0,249.0,248.0,0.0,30.0,31.0,31.0,15.0,46.0
1,Angry,0,0,0,0,0,0,0,0,0,...,241.0,112.0,208.0,225.0,98.0,48.0,60.0,253.0,251.0,225.0
2,Angry,0,0,0,0,0,0,0,0,0,...,151.0,255.0,83.0,14.0,207.0,0.0,22.0,236.0,0.0,4.0
3,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angry,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df = df.fillna(0)
df

,0,1,2,3,4,5,6,7,8,9,...,11202,11203,11204,11205,11206,11207,11208,11209,11210,11211
0,Angry,0,0,0,0,0,0,0,0,0,...,39.0,255.0,249.0,248.0,0.0,30.0,31.0,31.0,15.0,46.0
1,Angry,0,0,0,0,0,0,0,0,0,...,241.0,112.0,208.0,225.0,98.0,48.0,60.0,253.0,251.0,225.0
2,Angry,0,0,0,0,0,0,0,0,0,...,151.0,255.0,83.0,14.0,207.0,0.0,22.0,236.0,0.0,4.0
3,Angry,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Angry,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Sad,0,0,0,0,0,0,0,0,0,...,191.0,255.0,251.0,251.0,251.0,59.0,30.0,30.0,28.0,28.0
996,Sad,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,Sad,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,Sad,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df[0].value_counts()

0
Angry    250
Happy    250
Other    250
Sad      250
Name: count, dtype: int64

In [43]:
df.to_csv('dataset-ncdblbpac.csv')